# Simplificando el Análisis exploratorio de datos con python III: Limpieza de datos con pandas.

## Importar librerías.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats

## Cargar los datos a utilizar.

In [2]:
dir_prin = Path.cwd()
dir_entradas = dir_prin/'entradas'
dir_intermedios = dir_prin/'intermedios'
dir_salidas = dir_prin/'salidas'
df = pd.read_excel(dir_entradas/ 'clima2018_ZTCorrec.xlsx')

## Revisión inicial de los datos.

In [3]:
df.head()

,time,precip_intensity,temperature,apparent_temperature,dew_point,humidity,pressure,wind_speed,cloud_cover,dia,mes,hora
0,2018-01-01 00:00:00,0.0,22.44,22.73,18.10,0.77,NaN,0.0,0.19,lun,ene,0
1,2018-01-01 01:00:00,0.0,21.88,22.15,17.75,0.77,NaN,0.0,NaN,lun,ene,1
2,2018-01-01 02:00:00,0.0,21.33,21.63,17.94,0.81,NaN,0.0,NaN,lun,ene,2
3,2018-01-01 03:00:00,0.0,20.77,21.07,17.75,0.83,NaN,0.0,NaN,lun,ene,3
4,2018-01-01 04:00:00,0.0,19.11,19.33,16.79,0.86,NaN,0.0,NaN,lun,ene,4


## Descubriendo los datos faltantes.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8642 entries, 0 to 8641
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   time                  8642 non-null   datetime64[ns]
 1   precip_intensity      3361 non-null   float64       
 2   temperature           8640 non-null   float64       
 3   apparent_temperature  8640 non-null   float64       
 4   dew_point             8640 non-null   float64       
 5   humidity              8640 non-null   float64       
 6   pressure              98 non-null     float64       
 7   wind_speed            8306 non-null   float64       
 8   cloud_cover           6121 non-null   float64       
 9   dia                   8642 non-null   object        
 10  mes                   8642 non-null   object        
 11  hora                  8642 non-null   int64         
dtypes: datetime64[ns](1), float64(8), int64(1), object(2)
memory usage: 742.7+ K

In [5]:
df.isnull().sum().sort_values(ascending=False)

pressure                8544
precip_intensity        5281
cloud_cover             2521
wind_speed               336
humidity                   2
dew_point                  2
apparent_temperature       2
temperature                2
hora                       0
mes                        0
dia                        0
time                       0
dtype: int64

In [6]:
df[df['temperature'].isnull()]

,time,precip_intensity,temperature,apparent_temperature,dew_point,humidity,pressure,wind_speed,cloud_cover,dia,mes,hora
4727,2018-07-18 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mie,jul,5
8529,2018-12-27 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,7.55,0.19,jue,dic,4


## Eliminando las filas con datos faltantes.

In [7]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 6 to 7920
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   time                  58 non-null     datetime64[ns]
 1   precip_intensity      58 non-null     float64       
 2   temperature           58 non-null     float64       
 3   apparent_temperature  58 non-null     float64       
 4   dew_point             58 non-null     float64       
 5   humidity              58 non-null     float64       
 6   pressure              58 non-null     float64       
 7   wind_speed            58 non-null     float64       
 8   cloud_cover           58 non-null     float64       
 9   dia                   58 non-null     object        
 10  mes                   58 non-null     object        
 11  hora                  58 non-null     int64         
dtypes: datetime64[ns](1), float64(8), int64(1), object(2)
memory usage: 5.4+ KB


In [8]:
df.dropna().isnull().sum().sort_values(ascending=False)

hora                    0
mes                     0
dia                     0
cloud_cover             0
wind_speed              0
pressure                0
humidity                0
dew_point               0
apparent_temperature    0
temperature             0
precip_intensity        0
time                    0
dtype: int64

## Reemplazando los valores faltantes.

### Reemplazando con un escalar.

In [9]:
df.fillna(0).isnull().sum().sort_values(ascending=False)

hora                    0
mes                     0
dia                     0
cloud_cover             0
wind_speed              0
pressure                0
humidity                0
dew_point               0
apparent_temperature    0
temperature             0
precip_intensity        0
time                    0
dtype: int64

In [10]:
df[df['temperature'].fillna(0).isnull()]

,time,precip_intensity,temperature,apparent_temperature,dew_point,humidity,pressure,wind_speed,cloud_cover,dia,mes,hora


### Reemplazando con los métodos `pad` y `backfill`.

In [11]:
df.iloc[4726:4729]

,time,precip_intensity,temperature,apparent_temperature,dew_point,humidity,pressure,wind_speed,cloud_cover,dia,mes,hora
4726,2018-07-18 04:00:00,NaN,21.59,22.42,21.59,1.00,NaN,3.60,NaN,mie,jul,4
4727,2018-07-18 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mie,jul,5
4728,2018-07-18 06:00:00,NaN,22.47,23.10,20.59,0.89,NaN,11.15,0.19,mie,jul,6


In [12]:
df.fillna(method='backfill').iloc[4726:4729]

,time,precip_intensity,temperature,apparent_temperature,dew_point,humidity,pressure,wind_speed,cloud_cover,dia,mes,hora
4726,2018-07-18 04:00:00,0.0,21.59,22.42,21.59,1.00,1014.99,3.60,0.19,mie,jul,4
4727,2018-07-18 05:00:00,0.0,22.47,23.10,20.59,0.89,1014.99,11.15,0.19,mie,jul,5
4728,2018-07-18 06:00:00,0.0,22.47,23.10,20.59,0.89,1014.99,11.15,0.19,mie,jul,6


In [13]:
df.fillna(method='pad').iloc[4726:4729]

,time,precip_intensity,temperature,apparent_temperature,dew_point,humidity,pressure,wind_speed,cloud_cover,dia,mes,hora
4726,2018-07-18 04:00:00,0.0,21.59,22.42,21.59,1.00,1010.8,3.60,0.19,mie,jul,4
4727,2018-07-18 05:00:00,0.0,21.59,22.42,21.59,1.00,1010.8,3.60,0.19,mie,jul,5
4728,2018-07-18 06:00:00,0.0,22.47,23.10,20.59,0.89,1010.8,11.15,0.19,mie,jul,6


### Reemplazando con `groupby()` y `transform()`.

In [14]:
df.groupby('mes')['temperature'].transform(lambda grupo: grupo.fillna(np.mean(grupo))).iloc[4726:4729]

4726    21.590000
4727    25.615762
4728    22.470000
Name: temperature, dtype: float64

In [15]:
df[(df['mes'] == 'jul')]['temperature'].mean()

25.615761772853187

In [16]:
df.groupby('mes')['temperature'].transform(lambda grupo: grupo.fillna(np.median(grupo.dropna()))).iloc[4726:4729]

4726    21.59
4727    25.04
4728    22.47
Name: temperature, dtype: float64

In [17]:
np.median(df[(df['mes'] == 'jul')]['temperature'].dropna())

25.04

In [18]:
df.groupby('mes')['temperature'].transform(lambda grupo: grupo.fillna(stats.mode(grupo)[0][0])).iloc[4726:4729]

4726    21.59
4727    22.61
4728    22.47
Name: temperature, dtype: float64

In [28]:
stats.mode(df[(df['mes'] == 'jul')]['temperature'])[0][0]

22.61

In [20]:
df.groupby(['mes', 'hora'])['temperature'].transform(lambda grupo: grupo.fillna(np.mean(grupo))).iloc[4726:4729]

4726    21.59000
4727    21.63069
4728    22.47000
Name: temperature, dtype: float64

In [21]:
df.groupby(['mes', 'hora'])['temperature'].transform(lambda grupo: grupo.fillna(np.median(grupo.dropna()))).iloc[4726:4729]

4726    21.59
4727    21.59
4728    22.47
Name: temperature, dtype: float64

In [22]:
df.groupby(['mes', 'hora'])['temperature'].transform(lambda grupo: grupo.fillna(stats.mode(grupo)[0][0])).iloc[4726:4729]

4726    21.59
4727    21.59
4728    22.47
Name: temperature, dtype: float64

In [23]:
df['temperature'] = df.groupby(['mes', 'hora'])['temperature'].transform(lambda grupo: grupo.fillna(stats.mode(grupo)[0][0]))

In [24]:
df['apparent_temperature'] = df.groupby(['mes', 'hora'])['apparent_temperature'].transform(lambda grupo: grupo.fillna(stats.mode(grupo)[0][0]))

In [25]:
df['wind_speed'] = df.groupby(['mes', 'hora'])['wind_speed'].transform(lambda grupo: grupo.fillna(stats.mode(grupo)[0][0]))

In [26]:
df.isnull().sum().sort_values(ascending=False)

pressure                8544
precip_intensity        5281
cloud_cover             2521
humidity                   2
dew_point                  2
hora                       0
mes                        0
dia                        0
wind_speed                 0
apparent_temperature       0
temperature                0
time                       0
dtype: int64

In [27]:
df.to_excel(dir_salidas/ 'clima2018_Clean.xlsx', index=False)